<a href="https://colab.research.google.com/github/ValentinaEmili/Ethnicity-recognition/blob/main/majority_voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [339]:
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score

def merged_results(path):
  cnn_pred = pd.read_csv(f"{path}/CNN_predictions.csv")
  lightGBM_pred = pd.read_csv(f"{path}/LightGBM_predictions.csv")
  alexnet_pred = pd.read_csv(f"{path}/alexnet_predictions.csv")
  svm_pred = pd.read_csv(f"{path}/svm_predictions.csv")
  y_true = pd.read_csv(f"{path}/true_values.csv", index_col=False)

  cnn_pred["Image_Name"] = cnn_pred["Image_Name"].str.replace(".jpg", "", regex=False)
  lightGBM_pred["Image_Name"] = lightGBM_pred["Image_Name"].str.replace(".jpg", "", regex=False)
  alexnet_pred["Image_Name"] = alexnet_pred["Image_Name"].str.replace(".jpg", "", regex=False)
  svm_pred["Image_Name"] = svm_pred["Image_Name"].str.replace(".npy", "", regex=False)
  y_true["Image_Name"] = y_true["Image_Name"].str.replace(".npy", "", regex=False)

  cnn_pred = cnn_pred.set_index("Image_Name").sort_index()
  lightGBM_pred = lightGBM_pred.set_index("Image_Name").sort_index()
  alexnet_pred = alexnet_pred.set_index("Image_Name").sort_index()
  svm_pred = svm_pred.set_index("Image_Name").sort_index()
  y_true = y_true.set_index("Image_Name").sort_index()

  cnn_pred.rename(columns={"Predicted_Class": "Predicted_class_cnn"}, inplace=True)
  lightGBM_pred.rename(columns={"Predicted_Label": "Predicted_class_lightGBM"}, inplace=True)
  alexnet_pred.rename(columns={"Predicted_Class": "Predicted_class_alexnet"}, inplace=True)
  svm_pred.rename(columns={"Predicted_class": "Predicted_class_svm"}, inplace=True)

  merged = pd.concat([y_true, cnn_pred["Predicted_class_cnn"], lightGBM_pred["Predicted_class_lightGBM"], alexnet_pred["Predicted_class_alexnet"], svm_pred["Predicted_class_svm"]], axis=1)
  return merged

In [366]:
def get_prediction_eyes(row):
  counter = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
  counter[row["Predicted_class_cnn"]] += 0.3
  counter[row["Predicted_class_lightGBM"]] += 0.1
  counter[row["Predicted_class_alexnet"]] += 0.3
  counter[row["Predicted_class_svm"]] += 0.3
  return int(max(counter, key=counter.get))

In [304]:
def get_prediction_mouth(row):
  counter = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
  counter[row["Predicted_class_cnn"]] += 0.5
  counter[row["Predicted_class_lightGBM"]] += 0.2
  counter[row["Predicted_class_alexnet"]] += 0.4
  counter[row["Predicted_class_svm"]] += 0.4
  return int(max(counter, key=counter.get))

In [230]:
def get_prediction_nose(row):
  counter = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
  counter[row["Predicted_class_cnn"]] += 0.42
  counter[row["Predicted_class_lightGBM"]] += 0.1
  counter[row["Predicted_class_alexnet"]] += 0.5
  counter[row["Predicted_class_svm"]] += 0.6
  return int(max(counter, key=counter.get))

In [364]:
def get_final_prediction(row, feature):
  if feature == "eyes":
    return get_prediction_eyes(row)
  if feature == "mouth":
    return get_prediction_mouth(row)
  if feature == "nose":
    return get_prediction_nose(row)

In [365]:
def main():
  features = ["eyes", "mouth", "nose"]
  for feature in features:
    folder_path = f"/content/{feature}"
    merged = merged_results(folder_path)
    merged["Final_prediction"] = merged.apply(lambda row: get_final_prediction(row, feature), axis=1)
    print(f"feature: {feature}")
    print(accuracy_score(merged["True_class"], merged["Final_prediction"]))
    print(classification_report(merged["True_class"], merged["Final_prediction"]))

if __name__ == "__main__":
  main()

feature: eyes
0.748324958123953
              precision    recall  f1-score   support

           0       0.77      0.86      0.81      2018
           1       0.76      0.80      0.78       927
           2       0.84      0.74      0.79       670
           3       0.71      0.62      0.66       802
           4       0.42      0.29      0.34       359

    accuracy                           0.75      4776
   macro avg       0.70      0.66      0.68      4776
weighted avg       0.74      0.75      0.74      4776

feature: mouth
0.6802763819095478
              precision    recall  f1-score   support

           0       0.72      0.81      0.76      2018
           1       0.73      0.85      0.79       927
           2       0.58      0.50      0.54       670
           3       0.62      0.54      0.58       802
           4       0.41      0.16      0.23       359

    accuracy                           0.68      4776
   macro avg       0.61      0.57      0.58      4776
weighted av